# Model Training to suggest Alternative Queries for Bib Search Engine

## Read Data
Files are in gzipped JSON files and abstracts should be extracted. The following task are performed:

1. Import necessary libraries and modules such as os, gzip, json, pandas, re, nltk, gensim, and numpy.
2. Download stopwords and WordNet data from the NLTK library.
3. Define a function called extract_abstract_fields that takes a JSON data object as input and extracts the values of the "abstract" fields recursively from the JSON structure.
4. Define a function called read_gz_json that reads a gzipped JSON file, extracts the abstract fields using the extract_abstract_fields function, and returns a list of abstracts.
5. Specify the directory path where the JSON files are located.
6. Create an empty list called df_list.
7. Iterate over the files in the specified directory and if a file has the extension ".json.gz", read the file using the read_gz_json function and store the abstract fields in a DataFrame called abstract_df.
8. Append the abstract_df to the df_list.
9. Concatenate all the DataFrames in df_list into a single DataFrame called all_abstracts_df using the pd.concat function, ignoring the index.


In [9]:
import os
import gzip
import json
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string
import numpy as np


nltk.download('stopwords')
nltk.download('wordnet')


def extract_abstract_fields(json_data):
    abstract_fields = []

    def search_fields(data):
        if isinstance(data, dict):
            if "abstract" in data:
                abstract_fields.append(data["abstract"])
            for value in data.values():
                search_fields(value)
        elif isinstance(data, list):
            for item in data:
                search_fields(item)

    search_fields(json_data)
    return abstract_fields


def read_gz_json(file_name):
    with gzip.GzipFile(file_name, 'r') as fin:
        json_bytes = fin.read()
    json_str = json_bytes.decode('utf-8')
    data = json.loads(json_str)

    abstract_fields = extract_abstract_fields(data)

    return abstract_fields


dir_name = 'data/April22_CrossRef/'

df_list = []

for file in os.listdir(dir_name):
    if file.endswith('.json.gz'):
        file_path = os.path.join(dir_name, file)
        abstract_fields = read_gz_json(file_path)

        abstract_df = pd.DataFrame(abstract_fields, columns=['Abstract'])
        df_list.append(abstract_df)

all_abstracts_df = pd.concat(df_list, ignore_index=True)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rafaila/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/rafaila/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
all_abstracts_df.describe()

,Abstract
count,123116
unique,121721
top,<jats:p />
freq,351


## Data Preprocessing

These functions can be used to clean and preprocess text data by removing unnecessary characters, stopwords, and performing stemming or lemmatization to prepare the text for further analysis, such as text classification, sentiment analysis, or topic modeling. These are done using the Natural Language Toolkit (NLTK) library in Python.

In [11]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re
import string

# Create a set of English stopwords
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Convert to lower case
    text = text.lower()

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove specific characters
    text = text.replace('°', '').replace('..', '').replace(',′', '')

    # Remove 'ie', single letters, and numbers
    text = re.sub(r'\bie\b|\b[a-zA-Z]\b|\b[\d]+\b', '', text)

    # Tokenize
    words = word_tokenize(text)

    # Remove punctuation and stopwords, and strip trailing spaces
    words = [word for word in words if word not in string.punctuation and word not in stop_words and word.strip()]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return words

def preprocess_data(text):
    # Check if the input is a valid string or bytes-like object
    if not isinstance(text, (str, bytes)):
        raise ValueError("Input must be a string or bytes-like object.")

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Lowercase
    text = text.lower()
    # Tokenize
    words = word_tokenize(text)
    # Remove stopwords and stem
    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()
    words = [ps.stem(word) for word in words if word not in stop_words]
    return words

In [12]:
all_abstracts_df['cleaned_abstract'] = all_abstracts_df['Abstract'].apply(clean_text)

In [13]:
# Example usage
all_abstracts_df['cleaned_abstract'] = all_abstracts_df['cleaned_abstract'].apply(lambda x: preprocess_data(' '.join(x)))


In [14]:
all_abstracts_df.head()

,Abstract,cleaned_abstract
0,<jats:p> The use of reactive solvents offers a...,"[use, reactiv, solvent, offer, interest, flexi..."
1,<jats:p> Aromatic copolyimides were prepared u...,"[aromat, copolyimid, prepar, use, biphenyltetr..."
2,<jats:p> Phase separation in unreinforced reac...,"[phase, separ, unreinforc, reactiv, blend, stu..."
3,<jats:p> A new impregnating epoxide resin has ...,"[new, impregn, epoxid, resin, develop, optim, ..."
4,<jats:p>Marchuk's model of a general immune re...,"[marchuk, model, gener, immun, reaction, prese..."


## Text vectorization and KNN

The code you provided demonstrates a process of using the scikit-learn library in Python to perform text vectorization and nearest neighbors search. This code is commonly used for tasks such as document similarity or recommendation systems. Let's break down the code step by step:

#### 1. Processing the sentences:

The process_sentences function is called with input_df as the argument, and the result is assigned to the sentences variable.
This step cleans and tokenizes the sentences in the 'Abstract' column of the dataframe, storing the processed sentences in a list.

#### 2. Vectorizing sentences using TF-IDF:

The TfidfVectorizer class is instantiated as vectorizer.
The fit_transform method of the vectorizer is called with sentences as the argument.
This step converts the processed sentences into a matrix of TF-IDF features, where each row corresponds to a sentence, and each column represents a unique word or term in the sentences.

#### 3. Fitting a NearestNeighbors model:

The NearestNeighbors class is instantiated as model.
The fit method of the model is called with the TF-IDF matrix X as the argument.
This step fits the nearest neighbors model to the TF-IDF matrix.
Saving the model and vectorizer:

The pickle module is imported to save the model and vectorizer objects for later use.
The model object is saved to a file named 'knn_model.pkl'.
The vectorizer object is saved to a file named 'vectorizer.pkl'.
By following these steps, the code preprocesses text data, vectorizes it using TF-IDF, fits a nearest neighbors model to the vectorized data, and saves the model and vectorizer for future use.

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from spacy.lang.en.stop_words import STOP_WORDS

def tokenize(words):
    return [word for word in words if word not in STOP_WORDS]


def get_sentences(input_df):
    for index, row in input_df.iterrows():
        yield simple_preprocess(row['Abstract'])
        
def process_sentences(input_df):
    sentences = []
    for sentence in get_sentences(input_df):
        cleaned_sentence = clean_text(' '.join(sentence))
        tokenized_sentence = tokenize(cleaned_sentence)
        sentences.append(' '.join(tokenized_sentence))
    return sentences

input_df = all_abstracts_df
sentences = process_sentences(input_df)

# Convert sentences to a matrix of TF-IDF features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)

# Fit a NearestNeighbors model to the data
model = NearestNeighbors(n_neighbors=5)
model.fit(X)

# Save the model and vectorizer for later use
import pickle
with open('knn_model.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)


Use a trained nearest neighbors model and vectorizer to propose alternatives for a given query

In [19]:
def propose_alternatives(query, model, vectorizer):
    query_vector = vectorizer.transform([query])
    distances, indices = model.kneighbors(query_vector)
    return indices[0]

# Load the model and vectorizer
with open('knn_model.pkl', 'rb') as f:
    model = pickle.load(f)
with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

query = 'Machine learning for software engineering'
alternatives = propose_alternatives(query, model, vectorizer)

for i in alternatives:
    print(sentences[i])


jat software testing method executing program intension finding error process complex time intensive costly automation software testing process turn workable solution software engineering purpose machine learning algorithm chose primary research study tried explain machine learning based approach software testing overview input available data given model learning helping giving prediction discovered machine learning method employed test case development refining evaluation machine learning evaluate production test oracle predict cost testing procedure jat
jat order create software application necessary understand concept software engineering paper describe software engineering followed deliver software application paper architecture software discussed category architecture software pattern category architecture advantage challenge implementation software jat





## Word2Vec 

#### Help function 

The purpose of this callback is to track and display the training loss after each epoch, providing a measure of how the model is learning and improving over time. This information can be helpful for monitoring the training progress and assessing the model's performance.

In [14]:
import time
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()

        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss - self.loss_previous_step))

        self.epoch += 1
        self.loss_previous_step = loss


### Τaining a Word2Vec model using Gensim in Python

#### Importing necessary modules:

* The logging module is imported to enable logging functionality.
* The gensim.models module is imported to access the Word2Vec model class.
* The gensim.models.phrases module is imported to work with multi-word expressions using the Phrases and Phraser classes.

#### Training the Phrases model:

* The Phrases model is trained using the Phrases class on the 'cleaned_abstract' column of the all_abstracts_df dataframe.
* The min_count parameter sets the minimum frequency of a phrase to be considered.
* The threshold parameter sets the threshold score for forming a phrase.
* This step is used to connect multi-word expressions in the text data.

#### Training the Word2Vec model:

The sentences variable is assigned the transformed sentences obtained from applying the bigram model (Phrases model) on the 'cleaned_abstract' column of the all_abstracts_df dataframe.

The Word2Vec model is instantiated as model_simple with the desired parameters:
* sentences is the preprocessed sentences on which the model will be trained.
* vector_size is the dimensionality of the word vectors.
* window is the maximum distance between the target word and its context words.
* min_count is the minimum frequency threshold for a word to be included in the vocabulary.
* workers is the number of worker threads to train the model in parallel.
* compute_loss=True enables the computation of the training loss during the training process.
* callbacks=[callback()] specifies the callback function to track and print the loss after each epoch.

The train method of the model_simple object is called to train the model further.
The sentences are provided as input data.
total_examples is set to model_simple.corpus_count, which indicates the total number of examples (sentences) in the corpus.
epochs specifies the number of training iterations.
compute_loss=True enables the computation of the training loss during the training process.
callbacks=[callback()] specifies the callback function to track and print the loss after each epoch.


In [17]:
import logging 
from gensim.models import Word2Vec, Phrases
from gensim.models.phrases import Phraser

# Training the Phrases model to connect multi-word expressions
bigram = Phrases(all_abstracts_df['cleaned_abstract'].tolist(), min_count=2, threshold=5)

# Using the Phrases model for training word vectors
sentences = bigram[all_abstracts_df['cleaned_abstract'].tolist()] 

model_simple = Word2Vec(
    sentences,
    vector_size=170,
    window=9,
    min_count=10,
    workers=12,
    compute_loss=True,
    callbacks=[callback()]
)

start_time = time.time()
model_simple.train(
    sentences,
    total_examples=model_simple.corpus_count,
    epochs=100,
    compute_loss=True,
    callbacks=[callback()]
)
# Calculate and print the training time
end_time = time.time()
print("Training time: {}s".format(end_time - start_time))

# Save the model for later use
model_simple.save("word2vec1.model")


Loss after epoch 0: 2013547.125
Loss after epoch 1: 1704544.875
Loss after epoch 2: 1616776.0
Loss after epoch 3: 1428977.5
Loss after epoch 4: 1296910.5
Loss after epoch 0: 1610334.625
Loss after epoch 1: 1360190.625
Loss after epoch 2: 1534656.75
Loss after epoch 3: 1264835.0
Loss after epoch 4: 1159422.0
Loss after epoch 5: 1192438.0
Loss after epoch 6: 1054586.0
Loss after epoch 7: 1164941.0
Loss after epoch 8: 1057960.0
Loss after epoch 9: 1101458.0
Loss after epoch 10: 1007575.0
Loss after epoch 11: 1078382.0
Loss after epoch 12: 1009076.0
Loss after epoch 13: 1151640.0
Loss after epoch 14: 847713.0
Loss after epoch 15: 877336.0
Loss after epoch 16: 923292.0
Loss after epoch 17: 803480.0
Loss after epoch 18: 925674.0
Loss after epoch 19: 776512.0
Loss after epoch 20: 914674.0
Loss after epoch 21: 831320.0
Loss after epoch 22: 778494.0
Loss after epoch 23: 869056.0
Loss after epoch 24: 791802.0
Loss after epoch 25: 714756.0
Loss after epoch 26: 809846.0
Loss after epoch 27: 853928

Some basic metrics of our model.
1. Vocabulary Size: It gives the number of unique words in your model's vocabulary.
2. Embedding Size: It gives the dimensionality of the word vectors.
3. Sample Word Vector: It presents the word vector of a sample word, which helps to understand the structure of the word vectors.
4. Most Similar Words: For a given word, you can find the most similar words in the vocabulary. This can help to illustrate the semantic relationships learned by the model.

In [25]:

model_simple = Word2Vec.load("word2vec1.model")
# Get the vocabulary from your model
vocab = list(model_simple.wv.key_to_index)

# Print the vocabulary
print("Vocabulary Size: ", len(vocab))
print("Embedding Size: ", model_simple.vector_size)

import random

random_word = random.choice(model_simple.wv.index_to_key)
print("Random word from the vocabulary:", random_word)
print("Vector representation of '{}':\n{}".format(random_word, model_simple.wv[random_word]))


word = random.choice(model_simple.wv.index_to_key)
similar_words = model_simple.wv.most_similar(positive=[word], topn=5)
print("Words most similar to '{}':".format(word))
for word, similarity in similar_words:
    print("{}: {:.2f}".format(word, similarity))



Vocabulary Size:  92634
Embedding Size:  170
Random word from the vocabulary: euler
Vector representation of 'euler':
[ 1.8463045  -0.06705415  0.2547172  -2.5437639  -0.28381264 -5.532329
  0.56176656  1.065374    1.1329423   1.8228225  -0.5914034   0.55090314
  2.0438173   0.08780122  0.43613076  1.0922183   1.7926271   1.3081071
 -0.4504095   1.1848265   0.8958641   1.0969577  -0.2637198  -2.4744334
  0.22984296  0.08409906 -2.0167239   0.86515456  0.09193444  2.2129707
 -0.29017285 -2.420264    1.0281972   2.2223926   0.74556214 -1.0457335
 -2.1293983   0.9427547   1.1440977  -4.6470985   0.5070479   0.9725183
  3.3835607   2.3239877   2.15208    -2.1644535   1.1496928  -0.66145366
 -1.4161856  -2.6236892   3.5877209   1.4379928  -0.34087843  3.0576997
  1.4140025   1.5161414   1.9283969  -0.6957571  -1.8991587   1.1379238
 -0.91082406  2.388824   -2.7143767  -1.8783319   0.28064957 -1.0657226
  0.85440105  2.1324694  -3.2595408   0.5635338   0.63588566 -0.37891176
 -1.6751862   0.

In [2]:
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser

# model_simple = Word2Vec.load("word2vec1.model")
def preprocess_data(input_phrase):
    # Lowercase the phrase and tokenize it
    processed_phrase = word_tokenize(input_phrase.lower())
    # Load phrases model
    phrases_model = Phraser.load('phrases_model.txt')
    # Apply phrases model to the tokenized phrase
    processed_phrase = phrases_model[processed_phrase]
    return processed_phrase

def generate_alternative_sentences(input_phrase, num_sentences=5, topn=5):
    # Preprocess the input_phrase
    processed_input = preprocess_data(input_phrase)

    # Get the most similar words for each word in the input_phrase
    similar_words = {word: [item[0] for item in model_simple.wv.most_similar(word, topn=topn)] for word in processed_input if word in model.wv}

    # Generate new sentences
    new_sentences = set()
    attempts = 0
    max_attempts = num_sentences * 10  # Arbitrary number of maximum attempts

    while len(new_sentences) < num_sentences and attempts < max_attempts:
        for i in range(topn):  # Iterate over topn most similar words
            new_sentence = []
            for word in processed_input:
                # Select the ith most similar word if it exists in the similar_words dict, else use the original word
                new_word = similar_words.get(word, [word])[i % len(similar_words.get(word, [word]))]  # Use modulo to prevent index errors
                new_sentence.append(new_word)
            new_sentences.add(' '.join(new_sentence))
            attempts += 1
            if len(new_sentences) >= num_sentences:
                break  # Exit if we have enough sentences

    return [sentence.replace('_', ' ') for sentence in new_sentences]
print(generate_alternative_sentences("machine learning for software testing"))

['data driven for tools validation', 'deep learning for computer tests', 'prediction for graphical interface evaluation', 'artificial intelligence for hardware test', 'algorithms for simulation software assessment']


In [4]:
model_simple = Word2Vec.load("word2vec1.model")

words = ["machine", "learning", "for", "software", "testing"]
for word in words:
    if word in model_simple.wv:
        print(f"'{word}' is in the model's vocabulary.")
    else:
        print(f"'{word}' is NOT in the model's vocabulary.")


'machine' is NOT in the model's vocabulary.
'learning' is NOT in the model's vocabulary.
'for' is NOT in the model's vocabulary.
'software' is NOT in the model's vocabulary.
'testing' is NOT in the model's vocabulary.


In [7]:
# Get the vocabulary from your model
vocab = list(model.wv.key_to_index)

# Print the vocabulary
print(vocab)


['jats', 'jats_italic', 'jats_title', 'jats_sec', 'study', 'results', 'based', 'patients', 'data', 'de', 'jats_sub', 'research', 'jats_bold', 'model', 'analysis', 'jats_sup', 'methods', 'use', 'development', 'system', 'abstract', 'method', 'different', 'time', 'new', 'process', 'treatment', 'found', 'compared', 'group', 'high', 'work', 'level', 'associated', 'information', 'approach', 'effect', 'management', 'studies', 'control', 'including', 'dan', 'related', 'performance', 'potential', 'lt', 'quality', 'number', 'role', 'increase', 'important', 'effects', 'increased', 'design', 'students', 'paper', 'total', 'showed', 'years', 'factors', 'test', 'impact', 'groups', 'social', 'developed', 'higher', 'value', 'levels', 'non', 'background', 'specific', 'systems', 'conclusion', 'changes', 'present', 'order', 'identified', 'low', 'type', 'observed', 'respectively', 'cases', 'current', 'que', 'need', 'significant', 'state', 'models', 'activity', 'conducted', 'covid', 'addition', 'proposed', 

In [39]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 9.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 kB 10.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 5.4 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 9.4 MB/s eta 0:00:00


In [5]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, AlbertTokenizer

# initialize the model
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# the sentence to be paraphrased
sentence = "Example sentence."

# encode the sentence using the tokenizer
inputs = tokenizer.encode(sentence, return_tensors='pt')

# generate paraphrases using the model
outputs = model.generate(inputs)

# decode the generated paraphrases
paraphrases = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(paraphrases)


TypeError: 'NoneType' object is not callable

In [24]:
import re
import gensim
from gensim.models.phrases import Phrases, Phraser
from spacy.lang.en.stop_words import STOP_WORDS
import logging

logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

def read_input(input_df):
    for i, row in input_df.iterrows():
        if i % 10000 == 0:
            print(row)
            logging.info(f"read {i} reviews")
        # do some pre-processing and return a list of words for each review text
        yield gensim.utils.simple_preprocess(row['Abstract'])


read_input(all_abstracts_df)

<generator object read_input at 0x7fab97ff0660>

In [29]:
import re
import logging
from gensim.models import Word2Vec, Phrases
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess
from spacy.lang.en.stop_words import STOP_WORDS

def get_sentences(input_df):
    for index, row in input_df.iterrows():
        yield simple_preprocess(row['Abstract'])

def clean_text(text):
    # Convert to lower case
    text = text.lower()
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove specific characters
    text = text.replace('°', '').replace('..', '').replace(',′', '')
    # Remove 'ie', single letters, and numbers
    text = re.sub(r'\bie\b|\b[a-zA-Z]\b|\b[\d]+\b', '', text)
    return text

def tokenize(sentence):
    return [token for token in sentence.split() if token not in STOP_WORDS]

def build_phrases(sentences):
    phrases = Phrases(sentences, min_count=5, threshold=7, progress_per=1000)
    return Phraser(phrases)

def sentence_to_bi_grams(phrases_model, sentence):
    return ' '.join(phrases_model[sentence])

def sentences_to_bi_grams(phrases_model, input_df):
    n_grams_sentences = []
    for sentence in get_sentences(input_df):
        cleaned_sentence = clean_text(' '.join(sentence))
        tokenized_sentence = tokenize(cleaned_sentence)
        parsed_sentence = sentence_to_bi_grams(phrases_model, tokenized_sentence)
        n_grams_sentences.append(parsed_sentence.split())
    return n_grams_sentences


input_df = all_abstracts_df
sentences = list(get_sentences(input_df))
phrases_model = build_phrases(sentences)
phrases_model.save('phrases_model.txt')
sentences_with_bi_grams = sentences_to_bi_grams(phrases_model, input_df)

model_advanced = Word2Vec(
    sentences_with_bi_grams,
    vector_size=170,   
    window=9,          
    min_count=10,      
    workers=12,
    compute_loss=True,
    callbacks=[callback()]
)

model_advanced.train(sentences_with_bi_grams, total_examples=model_advanced.corpus_count, epochs=100, compute_loss=True, callbacks=[callback()])

model_advanced.save("word2vec.model")




INFO - 17:19:25: collecting all words and their counts
INFO - 17:19:25: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 17:19:25: PROGRESS: at sentence #1000, processed 186237 words and 132156 word types
INFO - 17:19:25: PROGRESS: at sentence #2000, processed 396700 words and 248478 word types
INFO - 17:19:25: PROGRESS: at sentence #3000, processed 616740 words and 348103 word types
INFO - 17:19:25: PROGRESS: at sentence #4000, processed 822576 words and 440056 word types
INFO - 17:19:25: PROGRESS: at sentence #5000, processed 1022977 words and 524896 word types
INFO - 17:19:25: PROGRESS: at sentence #6000, processed 1215523 words and 602232 word types
INFO - 17:19:26: PROGRESS: at sentence #7000, processed 1408265 words and 675228 word types
INFO - 17:19:26: PROGRESS: at sentence #8000, processed 1595800 words and 745988 word types
INFO - 17:19:26: PROGRESS: at sentence #9000, processed 1785275 words and 815159 word types
INFO - 17:19:26: PROGRESS: at sentence #100

INFO - 17:19:35: PROGRESS: at sentence #87000, processed 15932949 words and 4370116 word types
INFO - 17:19:35: PROGRESS: at sentence #88000, processed 16117085 words and 4408425 word types
INFO - 17:19:36: PROGRESS: at sentence #89000, processed 16327249 words and 4451638 word types
INFO - 17:19:36: PROGRESS: at sentence #90000, processed 16528522 words and 4492031 word types
INFO - 17:19:36: PROGRESS: at sentence #91000, processed 16727177 words and 4536316 word types
INFO - 17:19:36: PROGRESS: at sentence #92000, processed 16921141 words and 4574862 word types
INFO - 17:19:36: PROGRESS: at sentence #93000, processed 17122873 words and 4613586 word types
INFO - 17:19:36: PROGRESS: at sentence #94000, processed 17306035 words and 4648903 word types
INFO - 17:19:36: PROGRESS: at sentence #95000, processed 17487492 words and 4684300 word types
INFO - 17:19:37: PROGRESS: at sentence #96000, processed 17658728 words and 4717235 word types
INFO - 17:19:37: PROGRESS: at sentence #97000, pro

INFO - 17:20:25: EPOCH 0 - PROGRESS: at 19.24% examples, 2152562 words/s, in_qsize 23, out_qsize 0
INFO - 17:20:26: EPOCH 0 - PROGRESS: at 38.62% examples, 2172634 words/s, in_qsize 23, out_qsize 0
INFO - 17:20:27: EPOCH 0 - PROGRESS: at 59.36% examples, 2196558 words/s, in_qsize 23, out_qsize 0
INFO - 17:20:28: EPOCH 0 - PROGRESS: at 80.23% examples, 2217097 words/s, in_qsize 22, out_qsize 1
INFO - 17:20:29: EPOCH 0: training on 12595541 raw words (11171098 effective words) took 5.0s, 2233116 effective words/s


Loss after epoch 0: 2070181.375


INFO - 17:20:30: EPOCH 1 - PROGRESS: at 19.22% examples, 2156670 words/s, in_qsize 22, out_qsize 1
INFO - 17:20:31: EPOCH 1 - PROGRESS: at 39.60% examples, 2225660 words/s, in_qsize 24, out_qsize 0
INFO - 17:20:32: EPOCH 1 - PROGRESS: at 60.28% examples, 2233609 words/s, in_qsize 23, out_qsize 0
INFO - 17:20:33: EPOCH 1 - PROGRESS: at 81.33% examples, 2250586 words/s, in_qsize 23, out_qsize 0
INFO - 17:20:34: EPOCH 1: training on 12595541 raw words (11170764 effective words) took 5.0s, 2252174 effective words/s


...

INFO - 17:30:29: EPOCH 98 - PROGRESS: at 16.15% examples, 1803090 words/s, in_qsize 20, out_qsize 3
INFO - 17:30:30: EPOCH 98 - PROGRESS: at 33.32% examples, 1855763 words/s, in_qsize 23, out_qsize 0
INFO - 17:30:31: EPOCH 98 - PROGRESS: at 50.72% examples, 1875963 words/s, in_qsize 23, out_qsize 0
INFO - 17:30:32: EPOCH 98 - PROGRESS: at 67.51% examples, 1872685 words/s, in_qsize 23, out_qsize 0
INFO - 17:30:33: EPOCH 98 - PROGRESS: at 84.90% examples, 1880195 words/s, in_qsize 23, out_qsize 0
INFO - 17:30:34: EPOCH 98: training on 12595541 raw words (11170505 effective words) took 5.9s, 1880742 effective words/s


Loss after epoch 98: 257840.0


INFO - 17:30:35: EPOCH 99 - PROGRESS: at 16.35% examples, 1822434 words/s, in_qsize 23, out_qsize 0
INFO - 17:30:36: EPOCH 99 - PROGRESS: at 33.27% examples, 1854693 words/s, in_qsize 22, out_qsize 1
INFO - 17:30:37: EPOCH 99 - PROGRESS: at 50.32% examples, 1865852 words/s, in_qsize 23, out_qsize 0
INFO - 17:30:38: EPOCH 99 - PROGRESS: at 67.22% examples, 1869723 words/s, in_qsize 23, out_qsize 0
INFO - 17:30:39: EPOCH 99 - PROGRESS: at 84.58% examples, 1877053 words/s, in_qsize 23, out_qsize 0
INFO - 17:30:40: EPOCH 99: training on 12595541 raw words (11170812 effective words) took 5.9s, 1882808 effective words/s
INFO - 17:30:40: Word2Vec lifecycle event {'msg': 'training on 1259554100 raw words (1117063374 effective words) took 588.7s, 1897507 effective words/s', 'datetime': '2023-06-08T17:30:40.695995', 'gensim': '4.2.0', 'python': '3.7.3 (default, Oct 31 2022, 14:04:00) \n[GCC 8.3.0]', 'platform': 'Linux-4.19.0-24-amd64-x86_64-with-debian-10.10', 'event': 'train'}
INFO - 17:30:40: 

Loss after epoch 99: 300960.0


INFO - 17:30:40: storing np array 'syn1neg' to word2vec.model.syn1neg.npy
INFO - 17:30:41: not storing attribute cum_table
INFO - 17:30:41: saved word2vec.model


In [6]:
import os
import gzip
import json
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string
import random
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

import random 
model_advanced = Word2Vec.load("word2vec.model")
def preprocess_data(input_phrase):
    # Lowercase the phrase and tokenize it
    processed_phrase = word_tokenize(input_phrase.lower())
    # Load phrases model
    phrases_model = Phraser.load('phrases_model.txt')
    # Apply phrases model to the tokenized phrase
    processed_phrase = phrases_model[processed_phrase]
    return processed_phrase

def generate_alternative_sentences(input_phrase, num_sentences=5):
    # Preprocess the input_phrase
    processed_input = preprocess_data(input_phrase)

    # Get the most similar words for each word in the input_phrase
    similar_words = {word: [item[0] for item in model_advanced.wv.most_similar(word, topn=5)] for word in processed_input if word in model_advanced.wv}

    # Generate new sentences
    new_sentences = []
    for _ in range(num_sentences):
        new_sentence = []
        for word in processed_input:
            # Randomly select a similar word if it exists in the similar_words dict, else use the original word
            new_word = random.choice(similar_words.get(word, [word]))
            new_sentence.append(new_word)
        new_sentences.append(' '.join(new_sentence))

    return new_sentences

print(generate_alternative_sentences("machine learning"))


['artificial_intelligence', 'deep_learning', 'algorithms', 'prediction', 'deep_learning']
